In [1]:
%load_ext autoreload
%autoreload 2
#tensorboard doesn't work with keras model callbacks! it seems, needs tf.keras
from preprocess import *
#import keras
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import (Dense, Dropout, Flatten, Convolution2D, GlobalAveragePooling2D, BatchNormalization, Flatten,
                          GlobalMaxPool2D, MaxPool2D, concatenate, Activation, Input)
from tensorflow.keras import losses, models, optimizers
from tensorflow.keras.activations import relu, softmax
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import TensorBoard
import time  
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

Using TensorFlow backend.


In [2]:
labels = get_labels('./data')  
num_classes = len(labels[0])
labels

(['shh', 'silence', 'ttt'], array([0, 1, 2]), array([[1., 0., 0.],
        [0., 1., 0.],
        [0., 0., 1.]], dtype=float32))

In [3]:
#len of longest .wav data
max_sample_length = max_sample_len()

In [4]:
save_spec_data_to_array()

Saving spectrograms of label - 'ttt': 100%|██████████| 229/229 [00:01<00:00, 190.45it/s]


In [5]:
X_train, X_test, y_train, y_test = get_train_test()  


In [6]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)
len(X_train)

(421, 128, 33)
(421,)
(282, 128, 33)
(282,)


421

In [7]:
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], X_train.shape[2], 1)
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], X_test.shape[2], 1)

y_train_hot = to_categorical(y_train)
y_test_hot = to_categorical(y_test)

In [8]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)
print(y_train_hot.shape)#2nd dim is the number of labels

(421, 128, 33, 1)
(421,)
(282, 128, 33, 1)
(282,)
(421, 3)


In [9]:
#this is done in kaggle tutorial... if we do this do we have to normalize inputs for prediction too? sure right?
mean = np.mean(X_train, axis=0)
std = np.std(X_train, axis=0)

X_train = (X_train - mean)/std
X_test = (X_test - mean)/std

In [10]:
X_test[:20]

array([[[[-1.1079783 ],
         [-0.62152505],
         [-0.58889484],
         ...,
         [ 1.4605029 ],
         [ 1.4033936 ],
         [ 0.84169436]],

        [[-0.23813267],
         [-0.44308943],
         [-0.47158659],
         ...,
         [ 1.5998737 ],
         [ 1.5462843 ],
         [ 0.8503454 ]],

        [[-0.22090648],
         [-0.4004413 ],
         [-0.45001847],
         ...,
         [ 1.2819482 ],
         [ 1.2678732 ],
         [ 0.6224366 ]],

        ...,

        [[-1.1158917 ],
         [-1.0337932 ],
         [-0.9292683 ],
         ...,
         [ 2.2837095 ],
         [ 2.1776128 ],
         [ 1.4857608 ]],

        [[-0.87750953],
         [-0.9771985 ],
         [-1.0367979 ],
         ...,
         [ 2.314654  ],
         [ 2.1202824 ],
         [ 1.3678461 ]],

        [[-0.7339898 ],
         [-0.8073064 ],
         [-0.87210655],
         ...,
         [ 2.3506005 ],
         [ 2.08753   ],
         [ 1.1982627 ]]],


       [[[-1.1496819 ],


In [11]:
#modified kaggle tut, https://www.kaggle.com/fizzbuzz/beginner-s-guide-to-audio-data
def get_2d_conv_model(training_shape):
    
    inp = Input(shape=(training_shape[1], training_shape[2], 1))
    x = Convolution2D(64, (4,10), padding="same")(inp)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)
    x = MaxPool2D()(x)
    
    x = Flatten()(x)
    x = Dense(128)(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)
    out = Dense(num_classes, activation=softmax)(x)

    model = models.Model(inputs=inp, outputs=out)
    opt = optimizers.Adam(0.001)#learning rate

    model.compile(optimizer=opt, loss=losses.categorical_crossentropy, metrics=['acc'])
    return model

# Building The Model Then Training it

In [12]:
#NAME = "{}-conv-{}-nodes-{}-dense-{}".format(conv_layer, layer_size, dense_layer, int(time.time()))
NAME = "specmodel{}".format(int(time.time()))
tensorboard_cb = TensorBoard(log_dir="logs/{}".format(NAME), histogram_freq=1)
%load_ext tensorboard
tensorboard_cb

In [13]:
model = get_2d_conv_model(X_train.shape)
model.fit(X_train, y_train_hot, batch_size=100, epochs=12, verbose=True, validation_data=(X_test, y_test_hot), callbacks=[tensorboard_cb])

Train on 421 samples, validate on 282 samples
Epoch 1/12
421/421 [==============================] - 3s 7ms/sample - loss: 0.8680 - acc: 0.6271 - val_loss: 1.7176 - val_acc: 0.7021
Epoch 2/12
421/421 [==============================] - 3s 6ms/sample - loss: 0.4634 - acc: 0.8504 - val_loss: 0.9922 - val_acc: 0.7128
Epoch 3/12
421/421 [==============================] - 3s 6ms/sample - loss: 0.3282 - acc: 0.9026 - val_loss: 0.6908 - val_acc: 0.7270
Epoch 4/12
421/421 [==============================] - 3s 6ms/sample - loss: 0.2534 - acc: 0.9287 - val_loss: 0.5041 - val_acc: 0.7943
Epoch 5/12
421/421 [==============================] - 3s 6ms/sample - loss: 0.1980 - acc: 0.9430 - val_loss: 0.3968 - val_acc: 0.8582
Epoch 6/12
421/421 [==============================] - 3s 6ms/sample - loss: 0.1596 - acc: 0.9667 - val_loss: 0.3324 - val_acc: 0.8865
Epoch 7/12
421/421 [==============================] - 3s 6ms/sample - loss: 0.1201 - acc: 0.9691 - val_loss: 0.2879 - val_acc: 0.9078
Epoch 8/12
421/4

In [ ]:
%tensorboard --logdir logs/ --bind_all

In [16]:
#save the model and stats if you want to use it from the udp listener
save_model(model, NAME)
np.save("stats/{}.npy".format(NAME), [std, mean]) 
NAME

'specmodel1584403383'

## Prediction

In [15]:
def prep_data(filepath, shape):
    sample = wav2spec(filepath, max_sample_length=max_sample_length)
    sample = sample.reshape(1, shape[1], shape[2], 1)
    sample = (sample - mean)/std
    return sample

# Predicts one sample
def predict(filepath, model):
    sample_reshaped = prep_data(filepath, model.input_shape)
    return get_labels()[0][
            np.argmax(model.predict(sample_reshaped))
    ]

print(predict('./data/shh/W9M3W3YR15.wav', model=model))

shh
